In [ ]:
import requests
import configparser
import os

In [ ]:
config = configparser.ConfigParser()

# Ścieżka do pliku konfiguracyjnego
config_file_path = os.path.join(os.getcwd(), "config.ini")
config.read(config_file_path)

try:
    OPENROUTER_API_KEY = config['API']['OPENROUTER_API_KEY']
    PATH_SICK = config['Paths']['PATH_SICK']
    PATH_CONTROL = config['Paths']['PATH_CONTROL']
    MODEL_1 = config['Models']['MODEL_1']
    MODEL_2 = config['Models']['MODEL_2']
    print("Plik konfiguracyjny pomyślnie wczytany.")
except KeyError as e:
    print(f"Błąd: Brak klucza konfiguracyjnego: {e}\nUżywam ustawień domyślnych.")
    OPENROUTER_API_KEY = None
    # Ustawienia domyślne, dla jakich tworzono ten skrypt
    PATH_SICK = "data/descriptions_sick.xlsx"
    PATH_CONTROL = "data/descriptions_control_group.xlsx"
    MODEL_1 = "google/gemini-2.0-flash-exp:free"
    MODEL_2 = "deepseek/deepseek-chat-v3-0324:free"

In [ ]:
# Wczytaj plik z lokalizacji
with open("results/full_m1.txt", "r", encoding="utf-8") as f:
    model1_content = f.read()

with open("results/full_m2.txt", "r", encoding="utf-8") as f:
    model2_content = f.read()

# Prompt do modelu
prompt = f"""
Porównaj wyniki diagnozy HAE zawarte w dwóch poniższych plikach wykonanych przez dwa modele: {MODEL_1} oraz {MODEL_2}.
Powiedz, jakie informacje są zawarte w obu plikach.
Powiedz jakie są różnice: co występuje w analizie {MODEL_1}, czego nie ma w {MODEL_2} i odwrotnie.
Zaznacz wyraźnie, jakie informacje są z jakiego źródła. Nie bierz pod uwagę innych źródeł.

INFORMACJE Z PLIKU GEMINI:
{model1_content}

INFORMACJE Z PLIKU DEEPSEEK:
{model2_content}
"""

In [ ]:
# Wysyłanie zapytania do OpenRouter
response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": MODEL_1,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
)

# Wyświetl odpowiedź
print(response.json()["choices"][0]["message"]["content"])

### **Porównanie wyników diagnozy HAE w plikach Gemini i DeepSeek**  

#### **Podobieństwa (wspólne informacje w obu źródłach):**  
1. **Nawracające obrzęki:**  
   - Lokalizacja: twarz, kończyny, jama brzuszna, krtań.  
   - Czas trwania: 24–72 godziny.  
   - Brak pokrzywki i świądu (**Gemini**: brak u wszystkich oprócz jednego przypadku; **DeepSeek**: brak reakcji alergicznych).  
   - Nieskuteczność leków przeciwhistaminowych i glikokortykosteroidów.  

2. **Objawy żołądkowo-jelitowe:**  
   - Silne bóle brzucha, nudności, wymioty, wzdęcia.  
   - Mogą przypominać niedrożność jelit.  

3. **Czynniki wyzwalające:**  
   - Stres, urazy (w tym zabiegi stomatologiczne), infekcje.  

4. **Wywiad rodzinny:**  
   - Dziedziczny charakter choroby, występowanie podobnych objawów w rodzinie.  

5. **Badania laboratoryjne:**  
   - Obniżony poziom i aktywność inhibitora C1 esterazy (C1-INH).  
   - Niski poziom C4.  
   - Testy genetyczne (mutacja w *SERPING1* – w Gemini; brak nazwy genu w De

In [ ]:
# Save the response content to a file
with open("comparison/porownanie_modeli.txt", "w", encoding="utf-8") as file:
    file.write(response.json()["choices"][0]["message"]["content"])